# Save DNN model

In [8]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph()

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

INPUT_NODE = 784

LAYER1_NODE = 128
LAYER2_NODE = 64
LAYER3_NODE = 10


x = tf.placeholder(tf.float32, [None, 784], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='y')

W1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
b1 = tf.Variable(tf.truncated_normal([LAYER1_NODE], stddev=0.1))
W2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, LAYER2_NODE], stddev=0.1))
b2 = tf.Variable(tf.truncated_normal([LAYER2_NODE], stddev=0.1))
W3 = tf.Variable(tf.truncated_normal([LAYER2_NODE, LAYER3_NODE], stddev=0.1))
b3 = tf.Variable(tf.truncated_normal([LAYER3_NODE], stddev=0.1))

layer_1 = tf.matmul(x, W1) + b1
out1 = tf.nn.leaky_relu(layer_1, alpha=0.2)
layer_2 = tf.matmul(out1, W2) + b2
out2 = tf.nn.leaky_relu(layer_2, alpha=0.2)
layer_3 = tf.matmul(out2, W3) + b3
out3 = tf.nn.leaky_relu(layer_3, alpha=0.2)

y_predict = out3

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_predict))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name="accuracy")
print(accuracy)
saver = tf.train.Saver()

with tf.Session() as sess:
   
    sess.run(tf.global_variables_initializer())

    for step in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)

        train_step_, cross_entropy_ =sess.run([train_step, cross_entropy], feed_dict={x: batch_xs, y: batch_ys})
        if step % 50 == 0:
            print("step {}: cross_entropy is {}".format(step, cross_entropy_))
    accuracy_ = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
    
    saver.save(sess, "Saved_model/model.ckpt")
    
    print('Testing...... accuracy is {}'.format(accuracy_))


ModuleNotFoundError: No module named 'tensorflow.examples.tutorials'

# Restore DNN model (method 1)

In [11]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
tf.reset_default_graph()

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

INPUT_NODE =784

LAYER1_NODE = 128
LAYER2_NODE = 64
LAYER3_NODE = 10


x = tf.placeholder(tf.float32, [None, 784], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='y')

W1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
b1 = tf.Variable(tf.truncated_normal([LAYER1_NODE], stddev=0.1))
W2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, LAYER2_NODE], stddev=0.1))
b2 = tf.Variable(tf.truncated_normal([LAYER2_NODE], stddev=0.1))
W3 = tf.Variable(tf.truncated_normal([LAYER2_NODE, LAYER3_NODE], stddev=0.1))
b3 = tf.Variable(tf.truncated_normal([LAYER3_NODE], stddev=0.1))

layer_1 = tf.matmul(x, W1) + b1
out1 = tf.nn.leaky_relu(layer_1, alpha=0.2)
layer_2 = tf.matmul(out1, W2) + b2
out2 = tf.nn.leaky_relu(layer_2, alpha=0.2)
layer_3 = tf.matmul(out2, W3) + b3
out3 = tf.nn.leaky_relu(layer_3, alpha=0.2)

y_predict = out3

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_predict))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name="accuracy")
saver = tf.train.Saver()


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "Saved_model/model.ckpt")
    
    accuracy_ = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
    print('Testing...... accuracy is {}'.format(accuracy_))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from Saved_model/model.ckpt
Testing...... accuracy is 0.9577000141143799


# Restore DNN model (method 2)

In [10]:
import tensorflow as tf
tf.reset_default_graph()


saver = tf.train.import_meta_graph("Saved_model/model.ckpt.meta")       #關係圖

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, "Saved_model/model.ckpt")
    
    accuracy_ = sess.run("accuracy:0", feed_dict={"x:0": mnist.test.images, "y:0": mnist.test.labels})
    print('Testing...... accuracy is {}'.format(accuracy_))

INFO:tensorflow:Restoring parameters from Saved_model/model.ckpt
Testing...... accuracy is 0.9577000141143799


In [ ]:
#######預測蘑菇是否有毒########

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
header = ['class', 'cap_shape', 'cap_surface',
              'cap_color', 'bruises', 'odor', 'gill_attachment',
              'gill_spacing', 'gill_size', 'gill_color', 'stalk_shape',
              'stalk_root', 'stalk_surface_above_ring',
              'stalk_surface_below_ring', 'stalk_color_above_ring',
              'stalk_color_below_ring', 'veil_type', 'veil_color',
              'ring_number', 'ring_type', 'spore_print_color',
              'population', 'habitat']
df = pd.read_csv("agaricus-lepiota.data", sep=',', names=header)
df

,class,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [45]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

def prepare_data(data_file_name):
    """
    Responsible for cleaning the data file provided from the UCI machine
    learning repository here: http://archive.ics.uci.edu/ml/datasets/Mushroom.
    The function then produces two CSV files appropriately formatted to be
    used in TensorFlow where the CSV files split with respect to
    training and testing data.
    """

    # The header is formed from the 'agaricus-lepiota.name' file found on
    # http://archive.ics.uci.edu/ml/datasets/Mushroom
    header = ['class', 'cap_shape', 'cap_surface',
              'cap_color', 'bruises', 'odor', 'gill_attachment',
              'gill_spacing', 'gill_size', 'gill_color', 'stalk_shape',
              'stalk_root', 'stalk_surface_above_ring',
              'stalk_surface_below_ring', 'stalk_color_above_ring',
              'stalk_color_below_ring', 'veil_type', 'veil_color',
              'ring_number', 'ring_type', 'spore_print_color',
              'population', 'habitat']
    df = pd.read_csv(data_file_name, sep=',', names=header)
    #print (df)

    # Entries with a '?' indicate a missing piece of data, and
    # these entries are dropped from our dataset.
    df.replace('?', np.nan, inplace=True)
    df.dropna(inplace=True)

    # The class of poisonous or edible is indicated in the data as
    # either 'p' or 'e' respectively. We require that this is numeric,
    # and therefore use '0' to indicate poisonous (or not edible) and
    # '1' to indicate edible.
    df['class'].replace('p', 0, inplace=True)
    df['class'].replace('e', 1, inplace=True)

    # Since we are dealing with non-numeric feature data, or in other
    # words, categorical data, we need to replace these with numerical
    # equivalents. Pandas has a nice function called "get_dummies" that
    # performs this task.
    cols_to_transform = header[1:]
    df = pd.get_dummies(df, columns=cols_to_transform)                       #屬性都拆成one hot
    #print (df)
    # We can now split the data into two separate data frames,
    # one for training, which will constitute the bulk of the
    # data, and one for testing.
    df_train, df_test = train_test_split(df, test_size=0.1)

    # Determine the number of rows and columns in each of the
    # data frames.
    print (df_train.shape)
    print (df_test.shape)
    num_train_entries = df_train.shape[0]                                    #5079筆資料
    num_train_features = df_train.shape[1] - 1                               #99個屬性

    num_test_entries = df_test.shape[0]
    num_test_features = df_test.shape[1] - 1
    
    # The data frames are written as a temporary CSV file, as we still
    # need to modify the header row to include the number of rows and
    # columns present in the training and testing CSV files.
    df_train.to_csv('mushroom_train.csv', index=False)
    df_test.to_csv('mushroom_test.csv', index=False)
    
MUSHROOM_DATA_FILE = "agaricus-lepiota.data"

# Prepare the mushroom data for TensorFlow by
# creating two train / test CSV files.
prepare_data(MUSHROOM_DATA_FILE)



(5079, 99)
(565, 99)


In [40]:
df_train = pd.read_csv('mushroom_train.csv')
#print(df_train.head())

train_label = np.array(df_train['class'])
train_data =  np.array(df_train.drop('class', axis=1))
train_label

array([0, 1, 1, ..., 1, 0, 1])

In [41]:
df_test = pd.read_csv('mushroom_test.csv')
#print(df_test.head())
test_label = np.array(df_test['class'])
test_data =  np.array(df_test.drop('class', axis=1))

print(train_data.shape)

(5079, 98)


In [42]:
#solution
def one_hot(values):
    n_values = np.max(values) + 1
    return np.eye(n_values)[values]
train_label = one_hot(train_label.astype(int))
test_label = one_hot(test_label.astype(int))

print (train_label)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [43]:
import tensorflow as tf
tf.reset_default_graph()

batch_size = 100
INPUT_NODE = 98
LAYER1_NODE = 32
LAYER2_NODE = 2


x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x')
y = tf.placeholder(tf.float32, [None, LAYER2_NODE], name='y')

W1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
b1 = tf.Variable(tf.truncated_normal([LAYER1_NODE], stddev=0.1))
W2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, LAYER2_NODE], stddev=0.1))
b2 = tf.Variable(tf.truncated_normal([LAYER2_NODE], stddev=0.1))


layer_1 = tf.matmul(x, W1) + b1
out1 = tf.nn.leaky_relu(layer_1, alpha=0.2)
layer_2 = tf.matmul(out1, W2) + b2
out2 = tf.nn.leaky_relu(layer_2, alpha=0.2)


y_predict = out2

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=y_predict))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_predict, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name="accuracy")


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(1001):                                     #為何沒有step=step+batch_size?
        i = step
        if i+batch_size >= len(train_data):
                i = i+batch_size % len(train_data)
        batch_xs, batch_ys = train_data[i:i+batch_size], train_label[i:i+batch_size]
        
        train_step_, cross_entropy_ =sess.run([train_step, cross_entropy], feed_dict={x: batch_xs, y: batch_ys})
        if step % 50 == 0:
            print("step {}: cross_entropy is {}".format(step, cross_entropy_))
            
    accuracy_ = sess.run(accuracy, feed_dict={x: test_data, y: test_label})
    print('Testing...... accuracy is {}'.format(accuracy_))

step 0: cross_entropy is 0.6594077348709106
step 50: cross_entropy is 0.6101139187812805
step 100: cross_entropy is 0.5749460458755493
step 150: cross_entropy is 0.4900497496128082
step 200: cross_entropy is 0.4006577730178833
step 250: cross_entropy is 0.36826613545417786
step 300: cross_entropy is 0.35131990909576416
step 350: cross_entropy is 0.336247980594635
step 400: cross_entropy is 0.25668227672576904
step 450: cross_entropy is 0.20671872794628143
step 500: cross_entropy is 0.19451303780078888
step 550: cross_entropy is 0.17854303121566772
step 600: cross_entropy is 0.1629645675420761
step 650: cross_entropy is 0.15859925746917725
step 700: cross_entropy is 0.12463214248418808
step 750: cross_entropy is 0.11447204649448395
step 800: cross_entropy is 0.11282957345247269
step 850: cross_entropy is 0.08379519730806351
step 900: cross_entropy is 0.09613721072673798
step 950: cross_entropy is 0.08819933235645294
step 1000: cross_entropy is 0.05937248840928078
Testing...... accuracy 